In [12]:
import openai
import os

In [21]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

#process.env.API_KEY

openai.api_key  = os.getenv('API_KEY')

In [15]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [16]:
def get_completion_from_messages(messages, model = "gpt-3.5-turbo", temperature = 0):
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature ## degree of randomness
    )
    #print(response.choices[0].message) # to get the {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'} as output
    return response.choices[0].message["content"]

# Last chapter: Chat workflow

In [17]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [22]:
## helper function to get completion from the messages
response = get_completion_from_messages(messages, temperature=1) # higher temrature
print(response)

To prove to the possum that it could be done, forsooth!


In [31]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


In [32]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to your personal information, including your name. However, you can tell me your name, and I'd be happy to address you by it! How can I assist you today?


In [33]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa. How can I assist you further today?


## orderbot
 - We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant.

In [34]:
## collect user messages. collect prompt from teh user interface and than append to the list called context
## and then call the model with the context every time
def collect_messages(_): 
    '''this function work for collecting the messages'''
    
    prompt = inp.value_input ## enterd text from the user side in the input panel
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"}) ## collect the input prompt and appened in the context
    response = get_completion_from_messages(context) ## on the currnt context, get the response from the bot
    context.append({'role':'assistant', 'content':f"{response}"}) ## collect the bot response and append in the context
    
       ## design of teh dialogue line for user and assistant 
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [37]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)## previous conversations

dashboard = pn.Column(
    inp, ## text input
    pn.Row(button_conversation), ## button
    pn.panel(interactive_conversation, loading_indicator=True, height=300), ## all conversation history till last intercation
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [38]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "eggplant",
    "size": "6.75"
  },
  "toppings": [],
  "drinks": [
    {
      "type": "bottled water",
      "size": "small"
    }
  ],
  "sides": [],
  "total price": 7.75
}


#### write clear and specific instruction --- clear != short
#### give the model time to think

# Princinple 1: Write clear and specific instructions

## Tactic 1: using delimiters
##### Triple quotes: """"""
##### Triple backticks: ''''''
##### Triple dashes: ---
##### Angle brackets: <>
##### XML tags: '<tag></tag>'

In [12]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output,\
and reduce the chances of receiving irrelevant \
or incorrect response. Don't confuse writing a \
clear prompt with writing a short prmpt.\
In my case, longer prompts provide more clarity \
and context for the model, whiich can be lead to \
more detailed and relavent outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
````{text}```
"""
response = get_completion(prompt)
print(response)

It is important to provide clear and specific instructions to a model in order to guide it towards the desired output and reduce the chances of receiving irrelevant or incorrect responses, even if longer prompts are needed for clarity and context.


In [10]:
## Avoid prompt injection --- user give conflicting instructions


## Tactic 2: Ask for structured output
HTML, JSON

In [13]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre
"""
response = get_completion(prompt)
print(response)
## in python we can read this output into a dictionary or list

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Elena Rivers",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Past",
        "author": "Nathan Black",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Whispers in the Wind",
        "author": "Samantha Reed",
        "genre": "Romance"
    }
]


## Tactic 3: Check whether conditions are satisfied check assumptions required to do the task

In [12]:
text_1 = f"""
To brew a cup of tea, begin by boiling water.\
Once boiled, pour the hot water into a clean cup.\
Select your preferred tea bag or loose leaf tea and place it in the cup.\
Let the tea steep for about 3 to 5 minutes, depending on your desired strength.\
Use a spoon to press the tea bag gently against the side of the cup to release extra flavor.\
Remove the tea bag or strain the loose leaves. Optionally, add sweeteners or milk to taste.\
Stir and enjoy your freshly brewed cup of tea
"""
prompt_1 = f"""
You will be provided with text delimited by triple quotes.
If it contain a sequence of instructions, \
re-write those instructions in the following format:

Strp 1 - ...
Step 2 - ....
....
Step N: ....

If the text does not contain a sequence of instructions, \
then simply write \"Np steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt_1)
print("Completion for text 1: ")
print(response)

Completion for text 1: 
Step 1 - Begin by boiling water.
Step 2 - Once boiled, pour the hot water into a clean cup.
Step 3 - Select your preferred tea bag or loose leaf tea and place it in the cup.
Step 4 - Let the tea steep for about 3 to 5 minutes, depending on your desired strength.
Step 5 - Use a spoon to press the tea bag gently against the side of the cup to release extra flavor.
Step 6 - Remove the tea bag or strain the loose leaves.
Step 7 - Optionally, add sweeteners or milk to taste.
Step 8 - Stir and enjoy your freshly brewed cup of tea.


In [13]:
text_2 = f"""
Under a cloudless azure sky, the sun's radiant beams cascade down,\
warming the earth with golden hues. A gentle breeze carries the scent of blooming flowers,\
dancing amidst lush greenery. The air is alive with the chirping of birds and the buzzing of insects, \
while distant laughter echoes through serene surroundings. \
Trees sway gracefully, casting dappled shadows on sun-kissed paths.\
The world awakens to the embrace of sunlight, as nature's beauty flourishes \
in the enchanting ambiance of a perfect sunny day.
"""

prompt_2 = f"""
You will be provided with text delimited by triple quotes.
If it contain a sequence of instructions, \
re-write those instructions in the following format:

Strp 1 - ...
Step 2 - ....
....
Step N: ....

If the text does not contain a sequence of instructions, \
then simply write \"Np steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt_2)
print("Completion for text 2: ")
print(response)

Completion for text 2: 
No steps provided.


## Tactic 4: Few shot prompting
### Give successfull examples of completing tasks then ask model to perform task

In [14]:
prompt_3 = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the\
grandest symphony origibbates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

response = get_completion(prompt_3)
print(response)

<grandparent>: Resilience is like a tree that bends in the storm but does not break. It is the ability to bounce back from adversity, to persevere in the face of challenges, and to grow stronger through hardship. Just as a tree's roots anchor it firmly in the ground, resilience comes from a strong foundation of inner strength and determination. Remember, my child, that even the strongest storms eventually pass, and with resilience, you can weather any storm that comes your way.


# Principle 2: Give the model time to think

## Tactic 1: Specify the steps to complete the task
#### Step 1: ...
#### Steep 2: ...
#### .....
#### Step N: ...

In [17]:
text_4 = """
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop  \
well. As they climbed, Singing joyfully, misfortune \
struck-Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight .
"""

prompt_4 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Seperate your answers with line breaks.

Text: 
```{text_4}````
"""

response = get_completion(prompt_4)
print("Completion for prompt 4: ")
print(response)

Completion for prompt 4: 
1 - Jack and Jill go on a quest to fetch water from a well, but encounter misfortune along the way, yet remain adventurous and continue exploring.

2 - Jack et Jill partent en quête d'eau d'un puits, mais rencontrent des malheurs en chemin, mais restent aventureux et continuent d'explorer.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill partent en quête d'eau d'un puits, mais rencontrent des malheurs en chemin, mais restent aventureux et continuent d'explorer.",
  "num_names": 2
}


In [19]:
prompt_4_2 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into German.
3 - List each name in the summary.
4 - Output a json object that contains the following \
keys: german_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <List of names in German summary>
Output JSON: <json with summary and num_names>

Text to summarize: <{text_4}>
"""

response = get_completion(prompt_4_2)
print("Completion for prompt 4 and another text format: ")
print(response)

Completion for prompt 4 and another text format: 
Summary: Jack and Jill go on a quest to fetch water from a well on a hill, but encounter misfortune along the way, yet remain adventurous and continue exploring with delight.

Translation: Jack und Jill machen sich auf die Suche nach Wasser aus einem Brunnen auf einem Hügel, erleben jedoch unterwegs ein Missgeschick, bleiben aber abenteuerlustig und erkunden weiter mit Freude.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack und Jill machen sich auf die Suche nach Wasser aus einem Brunnen auf einem Hügel, erleben jedoch unterwegs ein Missgeschick, bleiben aber abenteuerlustig und erkunden weiter mit Freude.",
  "num_names": 2
}


## Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [20]:
prompt_5 = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt_5)
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.


In [21]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

The total cost for the first year of operations is calculated as follows:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


## Model Limitations
### Hallucination: Makes statements that sound plausible but are not true
### Reducing hallucination: First find relevant information, then answer the question based on the relavent information

In [30]:
prompt = f"""
Tell me about Xperia K by Sony
"""
response = get_completion(prompt)
print(response)

I'm sorry, but there is no information available about a Sony Xperia K model. It is possible that you may have mistaken the model name or it could be a concept or prototype that has not been released to the public. Sony's Xperia line of smartphones typically includes models such as the Xperia 1, Xperia 5, and Xperia 10. If you have any more specific details or questions, please let me know and I will do my best to assist you.


# Iterative prompt development process

### prompt guidlines:
   - Be clear and specific.
   - Analyze why result does not give desire output. 
   - Refine the idea and the prompt.
   - Repeat.

### Getting the marketing product description from product fact sheet

In [31]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

## Issue 1: The text is too long
 - limit the number of words/sentences/characters

In [32]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stylish and versatile mid-century inspired office chair, perfect for both home and business settings. This chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more.

Customize your chair with several options of shell color and base finishes to suit your personal style. Choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome.

Constructed with a 5-wheel plastic coated aluminum base, this chair features a pneumatic adjust for easy raise/lower action. The dimensions of the chair are as follows: width 53 cm, depth 51 cm, height 80 cm, seat height 44 cm, and seat depth 41 cm.

Choose between soft or hard-floor caster options and two seat foam densities: medium or high. You can also opt for armless or 8 position PU armrests for added comfort.

Made with high-quality

In [39]:
## for making output short
## specifies sentence, characters, words
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Available in various colors and finishes, with options for upholstery and armrests. Designed for both home and business use, with a durable construction and adjustable height feature. Made in Italy for quality assurance.


In [40]:
len(response.split(" "))

47

## Issue 2:  text focus on the wrong details
   - ask to focus on the aspects that are relevant to the intended audience

In [43]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile office chair, part of a stylish mid-century inspired collection. Choose from a variety of shell colors and base finishes to suit your space. Constructed with a durable aluminum base and high-density foam seat for comfort. Perfect for home or office use. Product IDs: SWC-100, SWC-110.


## Description needs a table dimensions
 - Ask it to extract information an dorganized it in the table

In [44]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

<div>
<p>This mid-century inspired office chair is a versatile addition to any space. With a durable 5-wheel plastic coated aluminum base and comfortable HD36 foam seat, this chair is perfect for both home and business settings. Available in a variety of colors and finishes to suit your style.</p>
<p>Product IDs: SWC-100, SWC-110</p>
</div>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Dimension</th>
    <th>Measurements (inches)</th>
  </tr>
  <tr>
    <td>WIDTH</td>
    <td>20.87"</td>
  </tr>
  <tr>
    <td>DEPTH</td>
    <td>20.08"</td>
  </tr>
  <tr>
    <td>HEIGHT</td>
    <td>31.50"</td>
  </tr>
  <tr>
    <td>SEAT HEIGHT</td>
    <td>17.32"</td>
  </tr>
  <tr>
    <td>SEAT DEPTH</td>
    <td>16.14"</td>
  </tr>
</table>


In [45]:
from IPython.display import display, HTML

In [47]:
display(HTML(response))

Dimension,Measurements (inches)
WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""


### for larger prompts
- refine prompts with a batch examples

# Summarizing

### Summarize with a word/sentence/character limit

In [6]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

In [14]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


Summary: 
Soft and cute panda plush toy loved by daughter, but smaller than expected for the price. Arrived early, allowing for personal enjoyment before gifting.


### summarize with a focus on shipping and delivary

In [15]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Summary: The panda plush toy is soft and cute, but smaller than expected for the price. However, shipping was fast, arriving a day early.


### summarize with a focus on price and value

In [16]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The panda plush toy is loved for its softness and cuteness, but some customers feel it's a bit small for the price.


### comment
 - summarize with a focus on the product department

In [17]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
product deparmtment, responsible for determining the \
quality of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Summary: 
Cute and soft panda plush toy, loved by daughter, but perceived as slightly small for the price paid. Arrived early.


## summarize multiple product reviews

In [18]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]



In [19]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 Summary: 
Adorable panda plush loved by daughter, soft and cute, but smaller than expected for the price. Arrived early. 

1 Summary: 
Lamp with storage, affordable, fast delivery, excellent customer service for missing parts. Great company. 

2 Impressive battery life, small brush head, good deal for $50, generic replacement heads available, leaves teeth feeling clean. 

3 Summary: Price fluctuations, quality concerns, motor issues after a year, but efficient for specific tasks. Brand loyalty noted. 



# Inferring
 - Infer sentiment and topics from product reviews and news articles

In [21]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [22]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is positive. The reviewer is satisfied with the lamp, the customer service, and the company in general.


In [23]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [24]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


### identify anger

In [25]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


### Extract product and company name from customer reviews

In [26]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


### Doing Multiple tasks at once

In [27]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


### Inferring topics

In [32]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

### Infer 5 topics

In [53]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Survey
2. Job satisfaction
3. NASA
4. Social Security Administration
5. Government pledge


In [54]:
response.split(sep = ',')

['1. Survey\n2. Job satisfaction\n3. NASA\n4. Social Security Administration\n5. Government pledge']

In [55]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [56]:
topic_list

['nasa',
 'local government',
 'engineering',
 'employee satisfaction',
 'federal government']

### make a news alert for certain topics

In [57]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

[1, 0, 0, 1, 1]


In [74]:
type(response)

str

In [78]:
#string_list = '[1, 0, 0, 1, 1]'

# Remove the brackets and split the string by comma
response_values = response.strip('[]').split(', ')
print(response_values)
# Convert each string value to an integer
response_integer = [int(value) for value in string_values]
print(response_integer)
# Print the resulting list with integer values
type(response_integer)

['1', '0', '0', '1', '1']
[1, 0, 0, 1, 1]


list

In [79]:
topic_dict = dict(zip(topic_list, response_integer))
topic_dict

{'nasa': 1,
 'local government': 0,
 'engineering': 0,
 'employee satisfaction': 1,
 'federal government': 1}

In [80]:
#topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}

if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


# Transforming
 - Translating to a different language

### Translation
   - ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [82]:
prompt = f"""
Translate the following English text to German: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hallo, ich möchte gerne einen Mixer bestellen.


In [83]:
prompt = f"""
Tell me which language this is: 
```Hallo, ich möchte gerne einen Mixer bestellen.```
"""
response = get_completion(prompt)
print(response)

German


In [84]:
prompt = f"""
Translate the following  text to Bengali and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

Bengali: আমি একটি বাস্কেটবল অর্ডার করতে চাই

Spanish: Quiero ordenar un balón de baloncesto

English: I want to order a basketball


In [85]:
prompt = f"""
Translate the following text to German in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: Möchten Sie ein Kissen bestellen?
Informal: Möchtest du ein Kissen bestellen?


### Universal Translator
 - Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [86]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [87]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (French): La performance du système est plus lente que d'habitude.
English: "The system performance is slower than usual."

Korean: "시스템 성능이 평소보다 느립니다." 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."

Korean: "내 모니터에는 불이 켜지지 않는 픽셀이 있습니다." 

Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Korean: 내 마우스가 작동하지 않습니다 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken
Korean: 제 Ctrl 키가 고장 났어요 

Original message (This is Chinese.): 我的屏幕在闪烁
English: My screen is flickering
Korean: 내 화면이 깜박거립니다 



### Tone Transformation
 - Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [88]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention the specifications of a standing lamp that I believe may be of interest to you. 

Sincerely,
Joe


### Format Conversion
 - ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [89]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<html>
<head>
  <title>Restaurant Employees</title>
</head>
<body>
  <table>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </table>
</body>
</html>


In [90]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


### Spellcheck/Grammar check.
- Here are some examples of common grammar and spelling problems and the LLM's response.

- To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [91]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
No errors found.
Their goes my freedom. There going to bring they’re suitcases.

No errors found.

Corrected version:
Their goes my freedom. There going to bring their suitcases.
You're going to need your notebook.
No errors found.
No errors found


In [92]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [93]:
from redlines import Redlines  ## need python >= 3.8

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

ModuleNotFoundError: No module named 'redlines'

In [94]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush toy for my daughter's birthday to prevent her from constantly borrowing mine from my room. As an adult, I can attest that pandas are universally loved, and this plush is no exception. Its irresistibly soft and cute design makes it the perfect companion for my daughter, who takes it everywhere she goes. However, upon closer inspection, I noticed that one of the ears is slightly lower than the other, which seems to be a manufacturing flaw rather than intentional asymmetry. Additionally, I found the size of the plush to be smaller than expected given its price point. I believe there are larger options available for the same cost. Despite these minor drawbacks, I was pleasantly surprised when the plush arrived a day earlier than anticipated, allowing me to enjoy its charm before gifting it to my daughter. Overall, while there are some room for improvement, this panda plush remains a delightful addition to our collection.

### Thanks to the following sites:

 https://writingprompts.com/bad-grammar-examples/

# Expanding
 -  generate customer service emails that are tailored to each customer's review.

## Customize the automated reply to a customer email

In [96]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [97]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your feedback with us. We are truly sorry to hear about your experience with the pricing changes and the quality of the product. We apologize for any inconvenience this may have caused you.

If you have any further concerns or would like to discuss this matter further, please feel free to reach out to our customer service team. They will be more than happy to assist you.

We appreciate your loyalty and feedback as it helps us improve our products and services for all our customers.

Thank you again for your review.

AI customer agent


### Remind the model to use details from the customer's email
 - higher temprature higher variety ( 0 - 1 )
 - for task that require reliability, predictability (temperature = 0 )
 - for task that require variety (temperature = 0.3, 0.7 ..... )

In [100]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your feedback with us. We are sorry to hear about your experience with the pricing changes and the decrease in quality of the product. We apologize for any inconvenience this may have caused you.

If you have any further concerns or would like to discuss this matter further, please feel free to reach out to our customer service team for assistance.

We appreciate your loyalty and feedback as it helps us improve our products and services for all our customers.

Thank you again for your review.

AI customer agent
